<html><font size=6><B>3η εργαστηριακή άσκηση</B><BR>
    <BR><B>Link prediction - Πρόβλεψη συνδέσμων</B></BR>
    </font></html>

Νικόλαος Παγώνας, el18175

In [163]:
AM = 3117890 #TODO: change!

Όλα τα imports

In [164]:
import networkx as nx
import numpy as np
import pandas as pd
from tabulate import tabulate

#TODO: suppress warnings

Κάποιες global ρυθμίσεις

Συνάρτηση που τυπώνει τα δεδομένα σε μορφή πίνακα

In [165]:
def table(headers, data):
	print(tabulate(data, headers, tablefmt="fancy_grid"))

<font size=4><b>1. Κατασκευή γράφου και προεργασία για την πρόβλεψη συνδέσμων</b></font>
    <li><b>1(i).</b> Χρησιμοποιήστε το dataset <a href="http://konect.cc/networks/dbpedia-similar/"> Similarities (DBpedia)</a>, για να κατασκευάσετε τον μη κατευθυνόμενο (undirected) γράφο $G$ <a href="https://dl.acm.org/doi/10.1145/2487788.2488173">[1]</a>,<a href="https://link.springer.com/chapter/10.1007/978-3-540-76298-0_52">[2]</a>. Οι κόμβοι του γράφου αναπαριστούν σελίδες της Wikipedia και οι ακμές σχέση ομοιότητας μεταξύ τους. Στην σελίδα <a href="http://konect.cc/networks/dbpedia-similar/"> KONECT (DBpedia)</a> μπορείτε να βρείτε πληροφορίες για τις ιδιότητες του γράφου.</a>

In [166]:
G = nx.read_edgelist(path="dbpedia-similar/out.dbpedia-similar", comments="%", nodetype=int)
# G = nx.read_adjlist(path="dbpedia-similar/out.dbpedia-similar", comments="%", nodetype=int)

<b><u>Υπόδειξη:</u></b> Μπορείτε να ελέγξετε προγραμματιστικά αν το dataset περιέχει διπλότυπα των ακμών με την ίδια ή αντίστροφη σειρά (π.χ. ακμές (a,b) και (b,a)), και να λάβετε υπ' όψιν σας αυτήν σας την παρατήρηση, όπου τυχόν χρειαστεί, στην ερώτηση 2.

In [167]:
candidates, duplicates = set(), set()

for edge in G.edges():
	edge = min(edge), max(edge)
	
	(duplicates if edge in candidates else candidates).add(edge)

print(f"""The graph {"contains" if duplicates else "doesn't contain"} duplicate edges""")

The graph doesn't contain duplicate edges


Όπως αναφέρθηκε επίσης στην παρουσίαση του εργαστηρίου, αφαιρούμε τα self-loops, αν υπάρχουν:

In [168]:
self_loops = list(nx.selfloop_edges(G))
print(f"Self-loops: {self_loops}")
# G.remove_edges_from(self_loops)

Self-loops: [(420, 420)]


 <li><b>1(ii).</b> Εμφανίστε τον αριθμό των κόμβων, τον αριθμό των ακμών, τον μέσο βαθμό των κόμβων και τον αριθμό των συνεκτικών συνιστωσών (connected components) για τον γράφο $G$.</li>

In [169]:
initial_node_count = G.number_of_nodes()
initial_edge_count = G.number_of_edges()
initial_average_degree = np.mean([degree for node, degree in G.degree()])
initial_connected_components = nx.number_connected_components(G)

headers = []
data = [
	["Number of nodes", initial_node_count],
	["Number of edges", initial_edge_count],
	["Average degree", initial_average_degree],
	["Number of connected components", initial_connected_components]
]

print("Initial Graph")
table(headers, data)

Initial Graph
╒════════════════════════════════╤═══════════╕
│ Number of nodes                │ 430       │
├────────────────────────────────┼───────────┤
│ Number of edges                │ 565       │
├────────────────────────────────┼───────────┤
│ Average degree                 │   2.62791 │
├────────────────────────────────┼───────────┤
│ Number of connected components │  95       │
╘════════════════════════════════╧═══════════╛


<li><b>1(iii). </b>Υπολογίστε το σύνολο των ασύνδετων ζευγών κόμβων (unconnected pairs) στον γράφο χρησιμοποιώντας τον πίνακα γειτνίασης (adjacency matrix). Δώστε επίσης τον αριθμό τους. Λάβετε υπ' όψιν σας ότι ο γράφος είναι μη κατευθυνόμενος και δεν χρειάζεται να διατρέξετε ολόκληρο τον πίνακα γειτνίασης.</li>

In [170]:
adj_matrix = nx.adjacency_matrix(G, nodelist=range(1, initial_node_count + 1)).A

unconnected_pairs = set()

for r in range(initial_node_count - 1):
	for c in range(r + 1, initial_node_count):
		if adj_matrix[r][c] == 0:
			unconnected_pairs.add((r + 1, c + 1))

print("Unconnected pairs:", *list(unconnected_pairs)[:30], "...")
print()
print(f"Number of unconnected pairs: {len(unconnected_pairs)}")

Unconnected pairs: (132, 268) (192, 319) (35, 335) (54, 385) (95, 386) (53, 160) (144, 278) (185, 279) (245, 330) (47, 345) (107, 396) (148, 397) (146, 324) (247, 376) (44, 47) (49, 391) (197, 289) (7, 165) (48, 166) (139, 284) (199, 335) (259, 386) (9, 211) (102, 402) (201, 381) (60, 176) (13, 330) (151, 294) (99, 104) (252, 346) ...

Number of unconnected pairs: 91671


/tmp/ipykernel_932/1299267641.py:1: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  adj_matrix = nx.adjacency_matrix(G, nodelist=range(1, initial_node_count + 1)).A


<li><b>1(iv).</b> Υπολογίστε το σύνολο των ακμών (removable edges) που μπορούν να αφαιρεθούν από τον γράφο χωρίς να υπάρχει κατάτμηση (splitting) του γράφου (δηλ. ο αριθμός των συνεκτικών συνιστωσών θα πρέπει να παραμένει αμετάβλητος). Υπόδειξη: Ελέγχετε επίσης ότι με αφαίρεση ακμής δεν μειώνεται ο αριθμός των κόμβων (εφόσον δεν υπάρχει ξεχωριστό αρχείο μόνο με κόμβους στο dataset). Δώστε επίσης τον αριθμό των ακμών (removable edges) που μπορούν να αφαιρεθούν υπό αυτές τις προϋποθέσεις.</li>

In [171]:
# # removable edges: check splitting and number of nodes

# removable_edges = set()

# copyG = G.copy()

# for u, v in copyG.edges():
# 	# u, v = min(u, v), max(u, v)

# 	copyG.remove_edge(u, v)

# 	# TODO: remove temporary fix for (420, 420)
# 	# if nx.number_connected_components(copyG) == initial_connected_components and len(copyG.nodes()) == initial_node_count and (u, v) != (420, 420):
# 	if nx.number_connected_components(copyG) == initial_connected_components and nx.number_of_isolates(copyG) <= 0 and (u, v) != (420, 420):
# 		removable_edges.add((u, v))
# 	else:
# 		copyG.add_edge(u, v)

# print("Removable edges:", *list(removable_edges)[:30], "...")
# print()
# print(f"Number of removable edges: {len(removable_edges)}")

# ren = removable_edges
# %store ren

lm, ln = [], []

def find_removable_edges(initial_G):
	''' Given a graph find which edges can be removed without splitting the graph.
	'''

	def is_split():
		''' checks if a graph is split.
		'''
		condition1 = (nx.number_of_isolates(G) > 0)
		condition2 = (nx.number_connected_components(G) !=
					  initial_connected_components)
		return (condition1 or condition2)

	removable_edges_right = set()
	G = initial_G.copy()

	DBPEDIA_SIMILAR_FILE = "./dbpedia-similar/out.dbpedia-similar"
	with open(DBPEDIA_SIMILAR_FILE, 'r') as f:
		for line in f:
			if line.startswith("%"):
				continue
			n1, n2 = [int(i) for i in line.split()]
			lm.append((n1, n2))
			G.remove_edge(n1, n2)
			if is_split():
				G.add_edge(n1, n2)
			else:
				removable_edges_right.add((n1, n2))
	
	removable_edges = set()
	G = initial_G.copy()
	keepedges = sorted(initial_G.edges(), key=lambda x: (x[0], x[1])).copy()

	keepedges = sorted([(min(u, v), max(u, v)) for u, v in keepedges])
	print("keepedges:", keepedges)

	for u, v in keepedges:
		ln.append((u, v))
		G.remove_edge(u, v)
		if is_split():
			G.add_edge(u, v)
		else:
			removable_edges.add((u, v))

	return removable_edges_right, removable_edges


removable_edges_right, removable_edges = find_removable_edges(G)

ren = removable_edges
rem = removable_edges_right

for m, n in zip(lm, ln):
	if m != n: print(m, n)

print()

for m, n in zip(lm, ln):
	print(m, n)

print((20, 32) in ln or (32, 20) in ln )

%store ren rem

keepedges: [(1, 2), (1, 3), (1, 4), (1, 5), (2, 3), (3, 4), (3, 5), (3, 6), (4, 6), (5, 6), (5, 46), (5, 47), (5, 48), (6, 32), (6, 46), (6, 47), (6, 48), (6, 49), (7, 8), (8, 9), (8, 10), (9, 168), (9, 184), (10, 185), (11, 12), (11, 13), (11, 14), (12, 14), (12, 15), (14, 16), (15, 113), (15, 161), (15, 162), (15, 163), (17, 18), (17, 19), (19, 20), (19, 21), (19, 22), (20, 21), (20, 23), (20, 24), (20, 25), (20, 26), (20, 27), (20, 28), (20, 29), (20, 30), (20, 31), (20, 32), (20, 33), (20, 34), (20, 35), (21, 22), (21, 30), (21, 31), (21, 32), (23, 24), (24, 34), (24, 35), (24, 36), (25, 35), (26, 37), (28, 32), (28, 38), (28, 39), (28, 40), (28, 41), (29, 68), (29, 89), (29, 90), (29, 100), (29, 101), (29, 102), (29, 104), (30, 74), (30, 157), (30, 158), (32, 42), (32, 54), (32, 147), (32, 251), (32, 312), (36, 147), (38, 41), (38, 73), (39, 74), (42, 43), (42, 44), (43, 44), (43, 45), (45, 154), (50, 51), (50, 52), (50, 53), (54, 55), (56, 57), (57, 58), (59, 60), (59, 61), (62, 

<li><b>1(va).</b> Κατασκευάστε dataframe που συνενώνει τις απαντήσεις των ερωτημάτων 1(iii) και 1(iv) (unconnected+removables). Χρησιμοποιήστε την μεταβλητή $link$ που θα παίρνει τιμή 1 ή 0 ανάλογα με την ύπαρξη συνδέσμου ή μη. To dataframe αυτό συγκεντρώνει τα θετικά και αρνητικά δείγματα για την πρόβλεψη των μελλοντικών συνδέσεων (link: target variable).

In [172]:
#dataframe of unconnected node pairs (they will function as negative samples)
unconnected_pairs_df = pd.DataFrame(data={"pair": sorted(unconnected_pairs)})
unconnected_pairs_df["link"] = 0 

#dataframe of removable edges (they will function as positive samples)
removable_edges_df = pd.DataFrame(data={"pair": sorted(removable_edges)})
removable_edges_df["link"] = 1  

#merge in one dataframe
merged = pd.concat(objs=[unconnected_pairs_df, removable_edges_df], ignore_index=True)

merged

,pair,link
0,"(1, 6)",0
1,"(1, 7)",0
2,"(1, 8)",0
3,"(1, 9)",0
4,"(1, 10)",0
...,...,...
91895,"(401, 403)",1
91896,"(401, 404)",1
91897,"(402, 403)",1
91898,"(402, 404)",1


<li><b>1(vb).</b> Κατασκευάστε τον γράφο $G'$ που προκύπτει από την αφαίρεση από τον γράφο $G$ των ακμών του ερωτήματος 1(iv).

In [173]:
#build graph G'=G_reduced

G_reduced = G.copy()

G_reduced.remove_edges_from(removable_edges)

<li><b>1(vc).</b> Εμφανίστε τον αριθμό των κόμβων, τον αριθμό των ακμών, τον μέσο βαθμό των κόμβων και τον αριθμό των συνεκτικών συνιστωσών (connected components) για τον γράφο $G'$.</li>

In [174]:
G_reduced_node_count = G_reduced.number_of_nodes()
G_reduced_edge_count = G_reduced.number_of_edges()
G_reduced_average_degree = np.mean([degree for node, degree in G_reduced.degree()])
G_reduced_connected_components = nx.number_connected_components(G_reduced)

headers = []
data = [
	["Number of nodes", G_reduced_node_count],
	["Number of edges", G_reduced_edge_count],
	["Average degree", G_reduced_average_degree],
	["Number of connected components", G_reduced_connected_components]
]

print("Graph after deleting removable edges")
table(headers, data)

Graph after deleting removable edges
╒════════════════════════════════╤═══════════╕
│ Number of nodes                │ 430       │
├────────────────────────────────┼───────────┤
│ Number of edges                │ 336       │
├────────────────────────────────┼───────────┤
│ Average degree                 │   1.56279 │
├────────────────────────────────┼───────────┤
│ Number of connected components │  95       │
╘════════════════════════════════╧═══════════╛


<font size=4><b>2. Εισαγωγή σε similarity-based μετρικές για Link prediction</b></font>
<ul>
	<li> Η μετρική <b>Jaccard Coefficient (JC)</b> των κόμβων ${u}$ και ${v}$ ορίζεται ως $\frac{|\Gamma(u) \cap \Gamma(v)|}{|\Gamma(u) \cup \Gamma(v)|}$, όπου ${\Gamma(u)}$ είναι το σύνολο των γειτονικών κόμβων του κόμβου ${u}$.</li>
	<li> Η μετρική <b>Preferential Αttachment (PA)</b> των κόμβων $u$ και $v$ ορίζεται ως $|\Gamma(u)| |\Gamma(v)|$, όπου $\Gamma(u)$ είναι το σύνολο των γειτονικών κόμβων του $u$.</li>
	<li> Η μετρική <b>Resource Allocation (RA)</b> των κόμβων $u$ και $v$ ορίζεται ως $\sum_{w \in \Gamma(u) \cap \Gamma(v)} \frac{1}{|\Gamma(w)|}$, όπου $\Gamma(u)$ είναι το σύνολο των γειτονικών κόμβων του $u$.</li>
</ul>


<li><B>2(i).</B> Για τον γράφο $G'$, που κατασκευάσατε στο ερώτημα 1(vβ), υπολογίστε την μετρική <B>Jaccard Coefficient (JC)</B>.  <u><a href="https://networkx.org/documentation/stable/reference/algorithms/link_prediction.html?highlight=link%20prediction">Χρησιμοποιήστε την υλοποιημένη συνάρτηση του λογισμικού NetworkX.</a></u></li>


In [175]:
jaccard_coefficients = list(nx.jaccard_coefficient(G_reduced))

print(len(G_reduced))
print(len(jaccard_coefficients))

430
91900


<li><b>2(ii). </b>Δώστε τις $k$ ακμές με την υψηλότερη τιμή μετρικής $JC$, όπου $k$ ο $Α.Μ.$ σας $mod$ $10$. Για $Α.Μ.$ λήγοντα σε $0$ δώστε τις $10$ ακμές με την υψηλότερη τιμή μετρικής. (Για την σύγκριση μεταξύ διαφορετικών μετρικών προτιμάται η εύρεση των $k$ επικρατέστερων συνδέσμων, αφού οι μετρικές επιστρέφουν διαφορετικό εύρος τιμών.)</li>


In [176]:
# k = Α.Μ. mod 10 
k = AM % 10 if AM % 10 != 0 else 10

topk_jaccard_coefficients = sorted(jaccard_coefficients, key=lambda item: item[2], reverse=True)[:k]

headers = [
	"Place", "Edge", "Jaccard coefficient"
]
data = [[i+1, (u, v), jaccard_coefficient] for i, (u, v, jaccard_coefficient) in enumerate(topk_jaccard_coefficients)]

table(headers, data)

╒═════════╤══════════╤═══════════════════════╕
│   Place │ Edge     │   Jaccard coefficient │
╞═════════╪══════════╪═══════════════════════╡
│       1 │ (4, 46)  │                     1 │
├─────────┼──────────┼───────────────────────┤
│       2 │ (4, 47)  │                     1 │
├─────────┼──────────┼───────────────────────┤
│       3 │ (4, 49)  │                     1 │
├─────────┼──────────┼───────────────────────┤
│       4 │ (23, 34) │                     1 │
├─────────┼──────────┼───────────────────────┤
│       5 │ (27, 33) │                     1 │
├─────────┼──────────┼───────────────────────┤
│       6 │ (46, 47) │                     1 │
├─────────┼──────────┼───────────────────────┤
│       7 │ (46, 49) │                     1 │
├─────────┼──────────┼───────────────────────┤
│       8 │ (47, 49) │                     1 │
├─────────┼──────────┼───────────────────────┤
│       9 │ (51, 52) │                     1 │
├─────────┼──────────┼───────────────────────┤
│      10 │ (

<li><b>2(iii). </b>Προσδιορίστε για την μέτρηση της απόδοσης της μετρικής $JC$ τα μεγέθη $Precision$, $Recall$ και $Accuracy$ και αναπαραστήστε τα γραφικά για τιμές κατωφλίου: $0.1$, $0.15$, $0.25$, $0.5$, $1.0$.  <br/><br/>
    $Precision=\frac{TP}{TP+FP}$, όπου $TP$: αληθώς θετικά αποτελέσματα, $FP$: ψευδώς θετικά αποτελέσματα <br/>
    $Recall=\frac{TP}{TP+FN}$, όπου $TP$: αληθώς θετικά αποτελέσματα, $FN$: ψευδώς αρνητικά αποτελέσματα</li> 
    $Accuracy=\frac{TP+TN}{TP+FN+TN+FP}$, όπου $TP$: αληθώς θετικά αποτελέσματα, $TN$: αληθώς αρνητικά αποτελέσματα, $FN$: ψευδώς αρνητικά αποτελέσματα, $FP$: ψευδώς θετικά αποτελέσματα</li><br/><br/>
        <b><u>Υπόδειξη:</u></b>
        Θετικό θεωρήστε ένα αποτέλεσμα, αν η μετρική $JC$ υπερβαίνει μια τιμή κατωφλίου (>=). Αρνητικό θεωρήστε το στην αντίθετη περίπτωση.
        <li>Αληθώς θετικά είναι τα αποτελέσματα που βγήκαν θετικά (η τιμή της $JC$ υπερβαίνει την τιμή κατωφλίου) και πράγματι θα υπάρξουν οι συνδέσεις που περιγράφουν (ερ. 1(iv)).</li>
        <li>Αληθώς αρνητικά είναι τα αποτελέσματα που βγήκαν αρνητικά και πράγματι δεν θα υπάρξουν οι συνδέσεις που περιγράφουν (ερ. 1(iii)).</li>
        <li>Ψευδώς θετικά είναι τα αποτελέσματα που βγήκαν θετικά, ενώ δεν θα υπάρξουν οι συνδέσεις που περιγράφουν.</li>
        <li>Ψευδώς αρνητικά είναι τα αποτελέσματα που βγήκαν αρνητικά, ενώ θα υπάρξουν οι συνδέσεις που περιγράφουν.</li>
        <li>Το άθροισμα $TN+FP$ πρέπει κάθε φορά να σας δίνει όλες τις ασύνδετες ακμές του γράφου $G$.</li>   
<li>Από το ερώτημα 1(iii) οι ακμές που υπολογίσατε είναι αυτές για τις οποίες δεν θα υπάρξουν συνδέσεις.</li> <li>Από το ερώτημα 1(iv) οι ακμές που υπολογίσατε είναι αυτές για τις οποίες θα υπάρξουν συνδέσεις.</li>

In [177]:
def JCfunction(threshold_jc):   
	results = []
	
	p, n, tp, tn, fp, fn = 0, 0, 0, 0, 0, 0

	for u, v, jc in jaccard_coefficients:
		# u, v = min(u, v), max(u, v)

		if jc >= threshold_jc:
			p += 1

			if (u, v) in removable_edges:
				tp += 1
			elif (u, v) not in removable_edges:
				fp += 1
		else:
			n += 1

			if (u, v) in unconnected_pairs:
				tn += 1
			elif (u, v) not in unconnected_pairs:
				fn += 1

	assert p == tp + fp
	assert n == tn + fn
	assert len(unconnected_pairs) == tn + fp, f"len(unconnected_pairs) = {len(unconnected_pairs)} != {tn + fp} = {tn} + {fp} = tn + fp"

	# print(tn, fp)
	print(fn, tp)

	precision_jc = tp / (tp + fp)
	recall_jc = tp / (tp + fn)
	accuracy_jc = (tp + tn) / (tp + fn + tn + fp)

	return precision_jc, recall_jc, accuracy_jc

metrics = []
for threshold_jc in [0.1, 0.15, 0.25, 0.5, 1.0]:
	precision_jc, recall_jc, accuracy_jc = JCfunction(threshold_jc)
	metrics.append({
		"threshold": threshold_jc,
		"precision": precision_jc,
		"recall": recall_jc,
		"accuracy": accuracy_jc
	})

metrics = pd.DataFrame(metrics)

67 162
69 160
73 156
90 139
102 127


In [12]:
upn = unconnected_pairs
# ren = removable_edges
# jcn = jaccard_coefficients

%store upn ren jcn

Stored 'upn' (set)


UsageError: Unknown variable 'ren'


<html><font size=4><b>3. Πρόβλεψη συνδέσμων βάσει similarity-based μετρικών</b></font>    
        <li><b>3(i).</b> Προτείνετε μια τουλάχιστον μέθοδο, για να εξερευνήσουμε και να εκφράσουμε μέσα από τα πακέτα NumPy ή SciPy ή Pandas την συσχέτιση (correlation) μεταξύ των τιμών των μετρικών <b>Jaccard Coefficient</b>, <b>Preferential Attachment</b>, <b>Resource Allocation</b> για τον μειωμένο γράφο $G'$. Σημ.: Δεν χρειάζεται κάποια υλοποίηση, μόνο αναφορά της μεθόδου συσχέτισης και επεξήγηση της επιλογής σας.
     <li><b>3(ii).</b> Εργαστείτε πάνω σε <b><u>αντίγραφο</u></b> του dataframe που κατασκευάσατε στο ερώτημα 1(vα) (unconnected+removables). Δημιουργήστε στήλη (column) με τις ακμές του γράφου και θέστε την ως index του dataframe. Στο dataframe θα προσθέσετε ως στήλες τις τιμές των μετρικών (<b>Jaccard Coefficient</b>, <b>Preferential Attachment</b>, <b>Resource Allocation</b>) για τον μειωμένο γράφο $G'$. Οι τιμές των μετρικών μπορούν να υπολογιστούν με τις συναρτήσεις του πακέτου NetworkX.</li></font>
    
|       | link  |Jacc.Coeff.|Pr.Attachment| Resource Allocation|
|-------|-------|-----------|-------------|--------------------|
|(1, 6) |  0    |  0.333333 |     32      |    0.700000        |
|(1, 46)|  0    |  0.200000 |      8      |    0.166667        |

In [5]:
#replicate of the dataframe "unconnected+removables"---dataframe will be used again in (4)

In [6]:
#dataframe for similarity-based measurement values
#Jaccard Coefficient

#Preferential Attachment

#Resource Allocation

<li><b>3(iiia).</b> Χρησιμοποιήστε τον Random Forest Classifier και το αντίγραφο dataframe που κατασκευάσατε στο ερώτημα 3(ii), για να προβλέψετε συνδέσεις. Συμπληρώστε στον κώδικα: τα features ($X$), την target variable ($y$) και τα ορίσματα της συνάρτησης $fit$ για το training του μοντέλου.</li>
<li><b>3(iiib).</b> Δώστε τις ($A.M$ $mod$ $10$) πρώτες προβλέψεις και την ακρίβεια ($accuracy$) του μοντέλου. </li>
<li><b>3(iiic).</b>Υπολογίστε την πιθανότητα να συνδεθούν οι κόμβοι ($1$,$47$) βάσει του μοντέλου σας. </li>  

In [7]:
#uncomment whole cell
#from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import accuracy_score
#from sklearn.model_selection import GridSearchCV
 
#X =             #features, uncomment and complete
#y = ...['link'] #target, uncomment and complete

#X_train, X_test, y_train, y_test = train_test_split(, , random_state = 0) #uncomment and complete

In [8]:
#uncomment whole cell
#classifier = RandomForestClassifier()   

#from sklearn.preprocessing import MinMaxScaler 
#mm_scaler = MinMaxScaler() 
#X_train_scaled = mm_scaler.fit_transform(X_train)
#X_test_scaled = mm_scaler.transform(X_test)

#parameters = {'n_estimators' : [10,50,100], 'max_depth' : [5,10,15]} 
#model = GridSearchCV(classifier, param_grid = parameters) 

In [9]:
#model.fit(X_train_scaled, )#train the model, uncomment and complete

In [10]:
#predictions = model.predict(X_test_scaled)       #uncomment
#predictions[ ] #get (A.M. last digit) first predictions, uncomment and complete

In [11]:
#predict_proba = model.predict_proba(...) #uncomment and complete

<html><font size=4><b>4. Πρόβλεψη συνδέσμων με embedding βασισμένο σε τυχαίους περιπάτους (Random Walks) </b></font><br/><br/>
    <li><b>4(i).</b> Εξάγετε τα χαρακτηριστικά (features) του γράφου $G'$, που προέκυψε από την αφαίρεση των ακμών. Για τον σκοπό αυτό χρησιμοποιήστε τον αλγόριθμο <b>Node2vec</b> <a href="https://cs.stanford.edu/~jure/pubs/node2vec-kdd16.pdf">[3]</a>. Ο αλγόριθμος αντιστοιχίζει τους κόμβους του γράφου μέσω τυχαίων περιπάτων (biased 2nd order random walks) σε ένα χώρο μικρού αριθμού διαστάσεων (low dimensional space) διατηρώντας την μεταξύ τους γειτνίαση.
        <br/>Για την επιλογή των παραμέτρων: 
            <li>p - παράμετρος τυχαίων περιπάτων p που καθορίζει την πιθανότητα “1/p” επιστροφής στον κόμβο προέλευσης (source node), </li>
                <li>q - παράμετρος τυχαίων περιπάτων q που καθορίζει την πιθανότητα  “1/q” μετακίνησης σε κόμβο μακριά από τον κόμβο προέλευσης (source node), </li>
                 <li>dimensions - πλήθος διαστάσεων των Node2Vec embeddings, </li>
                     <li>num_walks - αριθμός περιπάτων από κάθε κόμβο, </li>
                         <li>walk_length - μήκος τυχαίου περιπάτου, </li>
                             <li>window_size - μέγεθος παραθύρου context για τον αλγόριθμο Word2Vec, </li><li>num_iter - αριθμός SGD επαναλήψεων (epochs), </li>
            <li>workers - αριθμός workers για τον Word2Vec,</li>
            χρησιμοποιήστε τις τιμές:   </html>
   
                   

 |   p   |   q   | dimensions|  num_walks  |walk_length |window_size|        workers            |
 |-------|-------|-----------|-------------|------------|-----------|---------------------------|
 |  1.0  |  1.0  |  128      |     10      |    80      |     10    |multiprocessing.cpu_count()|

In [12]:
#install node2vec with conda / pip
#from node2vec import Node2Vec   #uncomment
#import multiprocessing          #uncomment

#generate walks
#node2vec_walker = Node2Vec(, , , , , , ) #uncomment and complete

#train node2vec model
#n2v_model = node2vec_walker.fit(window= , min_count=1) #uncomment and complete

<html><font>
    <li><b>4(ii).</b> Εφαρμόστε τον αλγόριθμο Random Forest Classifier, για να προβλέψετε τις συνδέσεις.</li><br/>
    <li><b>4(iii).</b> Υπολογίστε την ακρίβεια ($accuracy$) του μοντέλου. </li><br/></font></html>

In [13]:
#apply trained Node2Vec model on each node pair of the dataframe "unconnected+removables"
#uncomment:
#x = [(n2v_model.wv[str(i)]+n2v_model.wv[str(j)]) for i,j in zip(df_with_target_var['node_1'], df_with_target_var['node_2'])]

In [14]:
#X_train, X_test, y_train, y_test = train_test_split(, , random_state = 0)  #uncomment and complete

In [15]:
#uncomment whole cell
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import accuracy_score

#classifier = RandomForestClassifier(max_depth=2, random_state=0) 
#classifier.fit()   #uncomment and complete

<html><font size=4> <b><u>Ερώτηση bonus (+10/100)</u><b>: </font>Για τους binary operators για learning των edge features με τον αλγόριθμο Node2Vec που αναφέρονται στο <a href="https://cs.stanford.edu/~jure/pubs/node2vec-kdd16.pdf">[3]</a> ((a) Average, (b) Hadamard, (c) Weighted-L1 και (d) Weighted-L2) τροποποιήστε τoν κώδικά σας και σημειώστε τις παρατηρήσεις σας ως προς την απόδοση του αλγορίθμου.</font></html>

<html><font size=4><b><u>Βιβλιογραφία:</u></b><br/></font>
<a href="https://dl.acm.org/doi/10.1145/2487788.2488173">[1]	Jérôme Kunegis. KONECT – The Koblenz Network Collection. In Proc. Int. Conf. on World Wide Web Companion, pages 1343–1350, 2013.</a><br/>
<a href="https://link.springer.com/chapter/10.1007/978-3-540-76298-0_52">[2]	Sören Auer, Christian Bizer, Georgi Kobilarov, Jens Lehmann, Richard Cyganiak, and Zachary Ives. DBpedia: A nucleus for a web of open data. In Proc. Int. Semant. Web Conf., pages 722–735, 2008.<br/>
<a href="https://cs.stanford.edu/~jure/pubs/node2vec-kdd16.pdf">[3] A. Grover, J. Leskovec, Node2vec: Scalable Feature Learning for Networks. ACM KDD, 2016.</a><br/>
 <a href="https://proceedings.neurips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf">[4] T.Mikolov, I.Sutskever, K.Chen, G.S.Corrado and J.Dean, Distributed representations of words and phrases and their compositionality. In NIPS, 2013.</a><br/></font></html>